In [2]:
import pandas as pd
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise import SVD # importer ici les algo qu'on testera
from surprise import SVDpp
from surprise import KNNBasic
from surprise import SlopeOne
from surprise import model_selection
import random
import matplotlib.pyplot as plt

In [155]:
# Count the lines
num_lines = 1176952

# Sample size
size = int(num_lines/1)
 
# The row indices to skip - make sure 0 is not included to keep the header!
skip_idx = random.sample(range(1, num_lines), num_lines - size)
# Read the data
df = pd.read_csv("Datasets/data_train.csv", skiprows=skip_idx)

df[["user", "item"]] = df.Id.str.split("_", expand=True)

df.user = df.user.str.replace("r", "")
df.item = df.item.str.replace("c", "")
#########
#Might be uselfull to sample the test set just to see if the estimate is working
skip_idx = random.sample(range(1, num_lines), num_lines - size)
df2 = pd.read_csv("Datasets/sample_submission.csv", skiprows=skip_idx)
df2[["user", "item"]] = df2.Id.str.split("_", expand=True)
df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")
print(df.shape)
#del df

(1176952, 4)


In [156]:
occur_user=df.user.value_counts()
df_user_mod = df[~df.user.isin(occur_user[occur_user<=10].index)]
occur_item=df_user_mod.item.value_counts()
df_new=df_user_mod[~df_user_mod.item.isin(occur_item[occur_item<=10].index)]
reader = Reader(rating_scale=(1,5)) 
data = Dataset.load_from_df(df_new[["user","item","Prediction"]], reader)

In [116]:
df_new[df_new["user"]=="72"].mean()

Prediction    3.52
dtype: float64

In [101]:
tmp.user["72"]

IndexError: Column(s) user already selected

# Principe de SKLearn et Surprise qui est basé sur le premier:
* définition d'un test model à travers model_selection.gridsearchcv
* testmodel.fit qui va tester les model et paramètres pour retourner ce qu'il y a de mieux
* algorithm = testmodel.best_estimator["type de erreur choisi"] nous donne le meilleur algo selon les meilleurs paramètres
* algo.fit(data)
* algo.estimate(data a tester)

Dans la cell suivante, on va créer un "model selection" qui se base sur grid search cv. Ce dernier prend des paramètres et pourrait prendre plusieurs algos, a tester chacun selon les paramètres etc etc, mais ce n'est qu'une définition de model selection

Parameters pour SVD, pas forcément tous mais principalement :
* n_factors – The number of factors. Default is 100.
* n_epochs – The number of iteration of the SGD procedure. Default is 20.
* biased (bool) – Whether to use baselines (or biases). See note above. Default is True.
* init_mean – The mean of the normal distribution for factor vectors initialization. Default is 0.
* init_std_dev – The standard deviation of the normal distribution for factor vectors initialization. Default is 0.1.
* lr_all – The learning rate for all parameters. Default is 0.005.
* reg_all – The regularization term for all parameters. Default is 0.02.
* lr_bu – The learning rate for bu. Takes precedence over lr_all if set. Default is None.
* lr_bi – The learning rate for bi. Takes precedence over lr_all if set. Default is None.
* lr_pu – The learning rate for pu. Takes precedence over lr_all if set. Default is None.
* lr_qi – The learning rate for qi. Takes precedence over lr_all if set. Default is None.
* reg_bu – The regularization term for bu. Takes precedence over reg_all if set. Default is None.
* reg_bi – The regularization term for bi. Takes precedence over reg_all if set. Default is None.3
* reg_pu – The regularization term for pu. Takes precedence over reg_all if set. Default is None.
* reg_qi – The regularization term for qi. Takes precedence over reg_all if set. Default is None.
* random_state (int, RandomState instance from numpy, or None) – Determines the RNG that will be used for initialization. If int, random_state will be used as a seed for a new RNG. This is useful to get the same initialization over multiple calls to fit(). If RandomState instance, this same instance is used as RNG. If None, the current RNG from numpy is used. Default is None.
* verbose – If True, prints the current epoch. Default is False.m

In [157]:

svd=SVD(n_factors=100, n_epochs=100, biased=True, init_mean=0,
                 init_std_dev=.1, lr_all=0.001,
                 reg_all=0.06326530612244897, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None,
                 reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None,
                 random_state=None, verbose=True) 
svd.fit(data.build_full_trainset())

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [158]:
#Test with only svd :
df2 = pd.read_csv("Datasets/sample_submission.csv",skiprows=skip_idx)

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")

valid_item=df_new.item.unique()
valid_user=df_new.user.unique()

global_mean=df_new["Prediction"].mean()
count_bad_user=0
count_bad_item=0
count_bad_both=0
array_svd=np.ones((df2.shape[0],1))
for i in df2.iterrows():
    if (i[1]["user"] in valid_user and i[1]["item"] in valid_item):
        array_svd[i[0]]=svd.estimate(int(i[1][2]), int(i[1][3]))
    elif (i[1]["user"] in valid_user):
        array_svd[i[0]]=df_new[df_new["user"]==i[1]["user"]].mean()
        count_bad_user+=1
    elif (i[1]["item"] in valid_item):
        #take mean item
        array_svd[i[0]]=df_new[df_new["item"]==i[1]["item"]].mean()
        count_bad_item+=1
    else :
        #take global mean df_new or df ?
        array_svd[i[0]]=global_mean
        count_bad_both+=1
    if(i[0] % 1000==0):
        print(i[0])
        print(count_bad_user,count_bad_item,count_bad_both)

0
0 0 0
1000
0 0 0
2000
0 0 0
3000
0 0 0
4000
0 0 0
5000
0 0 0
6000
0 0 0
7000
0 1 0
8000
0 3 0
9000
0 4 0
10000
0 4 0
11000
0 5 0
12000
0 6 0
13000
0 7 0
14000
0 7 0
15000
0 7 0
16000
0 8 0
17000
0 8 0
18000
0 8 0
19000
0 8 0
20000
0 8 0
21000
0 9 0
22000
0 12 0
23000
0 12 0
24000
0 12 0
25000
0 12 0
26000
0 12 0
27000
0 12 0
28000
0 12 0
29000
0 12 0
30000
0 12 0
31000
0 12 0
32000
0 12 0
33000
0 12 0
34000
0 12 0
35000
0 12 0
36000
0 12 0
37000
0 14 0
38000
0 14 0
39000
0 14 0
40000
0 14 0
41000
0 14 0
42000
0 14 0
43000
0 14 0
44000
0 15 0
45000
0 15 0
46000
0 15 0
47000
0 15 0
48000
0 15 0
49000
0 15 0
50000
0 15 0
51000
0 15 0
52000
0 15 0
53000
0 15 0
54000
0 15 0
55000
0 15 0
56000
0 15 0
57000
0 17 0
58000
0 19 0
59000
0 19 0
60000
0 19 0
61000
0 19 0
62000
0 19 0
63000
0 20 0
64000
0 21 0
65000
0 21 0
66000
0 21 0
67000
0 21 0
68000
0 21 0
69000
0 21 0
70000
0 21 0
71000
0 22 0
72000
0 24 0
73000
0 24 0
74000
0 24 0
75000
0 25 0
76000
0 25 0
77000
0 25 0
78000
0 25 0
79000
0 

591000
0 114 0
592000
0 114 0
593000
0 114 0
594000
0 114 0
595000
0 114 0
596000
0 114 0
597000
0 114 0
598000
0 114 0
599000
0 114 0
600000
0 115 0
601000
0 115 0
602000
0 115 0
603000
0 116 0
604000
0 116 0
605000
0 117 0
606000
0 117 0
607000
0 117 0
608000
0 117 0
609000
0 117 0
610000
0 117 0
611000
0 117 0
612000
0 117 0
613000
0 117 0
614000
0 117 0
615000
0 117 0
616000
0 117 0
617000
0 117 0
618000
0 117 0
619000
0 117 0
620000
0 118 0
621000
0 118 0
622000
0 118 0
623000
0 118 0
624000
0 118 0
625000
0 119 0
626000
0 119 0
627000
0 119 0
628000
0 119 0
629000
0 119 0
630000
0 120 0
631000
0 120 0
632000
0 120 0
633000
0 120 0
634000
0 120 0
635000
0 120 0
636000
0 120 0
637000
0 120 0
638000
0 120 0
639000
0 120 0
640000
0 120 0
641000
0 121 0
642000
0 121 0
643000
0 121 0
644000
0 121 0
645000
0 121 0
646000
0 121 0
647000
0 122 0
648000
0 122 0
649000
0 122 0
650000
0 122 0
651000
0 122 0
652000
0 122 0
653000
0 122 0
654000
0 122 0
655000
0 122 0
656000
0 122 0
657000
0 1

1129000
0 172 0
1130000
0 172 0
1131000
0 172 0
1132000
0 172 0
1133000
0 172 0
1134000
0 172 0
1135000
0 172 0
1136000
9 172 0
1137000
9 172 0
1138000
9 172 0
1139000
9 172 0
1140000
9 172 0
1141000
9 172 0
1142000
9 172 0
1143000
9 172 0
1144000
9 172 0
1145000
9 172 0
1146000
9 172 0
1147000
9 172 0
1148000
9 172 0
1149000
9 172 0
1150000
9 172 0
1151000
9 172 0
1152000
9 172 0
1153000
9 172 0
1154000
9 172 0
1155000
9 172 0
1156000
9 172 0
1157000
9 172 0
1158000
9 172 0
1159000
9 172 0
1160000
9 172 0
1161000
9 172 0
1162000
9 172 0
1163000
9 172 0
1164000
9 172 0
1165000
9 172 0
1166000
9 172 0
1167000
9 172 0
1168000
9 172 0
1169000
9 172 0
1170000
9 172 0
1171000
9 172 0
1172000
9 172 0
1173000
9 172 0
1174000
9 172 0
1175000
9 172 0
1176000
9 172 0


In [164]:
array_svd_round=np.rint(array_svd)
print(np.unique(array_svd_round))
array_svd_round[np.where(array_svd_round>5)]=5
print(np.unique(array_svd_round))

[1. 2. 3. 4. 5. 6.]
[1. 2. 3. 4. 5.]


-139


In [ ]:
svdpp=SVDpp(n_factors=20, n_epochs=20, init_mean=0, init_std_dev=.1,
                 lr_all=.007, reg_all=.02, lr_bu=None, lr_bi=None, lr_pu=None,
                 lr_qi=None, lr_yj=None, reg_bu=None, reg_bi=None, reg_pu=None,
                 reg_qi=None, reg_yj=None, random_state=None, verbose=True)
svdpp.fit(data.build_full_trainset())

In [78]:
knn=KNNBasic(k=40, min_k=1, sim_options={}, verbose=True)
knn.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


In [79]:
sl=SlopeOne()
sl.fit(data.build_full_trainset())

In [68]:
#Reload this shit every time because we modify it after
df2 = pd.read_csv("Datasets/sample_submission.csv",skiprows=skip_idx)

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")

array_svd = np.ones((df2.shape[0],1))
array_svdpp = np.ones((df2.shape[0],1))
array_slope = np.ones((df2.shape[0],1))
array_knn = np.ones((df2.shape[0],1))
fails_slope=0
fails_knn=0
for i in df2.iterrows():
    array_svd[i[0]]=svd.estimate(int(i[1][2]), int(i[1][3]))
    array_svdpp[i[0]]=svdpp.estimate(int(i[1][2]), int(i[1][3]))
    """
    try:
        array_slope[i[0]]=sl.estimate(int(i[1][2]), int(i[1][3]))
    except Exception as e: 
        #print(e)
        fails_slope+=1
        array_slope[i[0]]=-1000
    try:
        array_knn[i[0]]=knn.estimate(int(i[1][2]), int(i[1][3]))[0]
    except Exception as e: 
        print(e)
        fails_knn+=1
        array_knn[i[0]]=-1000
    """
    if i[0]%10000==0:
        print(i[0])
print("Slope done with : "+ str(fails_slope)+" failures")    
print("KNN done with : "+ str(fails_knn)+" failures")

NameError: name 'svdpp' is not defined

In [17]:
all_pred=np.concatenate((array_svd,array_svdpp,array_slope,array_knn), axis=1)
print(np.unique(array_svd))
print(np.unique(array_svdpp))
print(np.unique(array_slope))
print(np.unique(array_knn))


for i in range (int(len(all_pred)/1)):
    print(all_pred[i])

[3.57113564 3.60305256 3.60327641 3.66461016 3.66587867 3.66624184
 3.6690817  3.71794872 3.73906428 3.74795428 3.75735846 3.82205957
 3.82678478]
[3.50991966 3.55000938 3.5556465  3.63119519 3.63264003 3.63346305
 3.64770821 3.71794872 3.7457118  3.74737442 3.74919942 3.87156192
 3.87332986]
[1.]
[1.]
[3.66624184 3.63346305 1.         1.        ]
[3.66624184 3.63346305 1.         1.        ]
[3.73906428 3.74737442 1.         1.        ]
[3.66587867 3.63119519 1.         1.        ]
[3.75735846 3.7457118  1.         1.        ]
[3.60305256 3.55000938 1.         1.        ]
[3.57113564 3.50991966 1.         1.        ]
[3.66461016 3.63264003 1.         1.        ]
[3.60327641 3.5556465  1.         1.        ]
[3.6690817  3.64770821 1.         1.        ]
[3.74795428 3.74919942 1.         1.        ]
[3.82678478 3.87156192 1.         1.        ]
[3.71794872 3.71794872 1.         1.        ]
[3.71794872 3.71794872 1.         1.        ]
[3.71794872 3.71794872 1.         1.        ]
[3.717

In [91]:
all_pred_rounded=np.rint(all_pred)
for i in range (int(len(all_pred_rounded)/1000)):
    print(all_pred_rounded[i])

[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 3.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 3.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 4.]
[3. 3. 4.]
[4. 3. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 3.]
[4. 4. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 3.]
[4. 3. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 4.]
[4. 3. 4.]
[4. 4. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 4.]
[3. 3. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 3. 3.]
[4. 3. 4.]
[4. 4. 4.]
[4. 3. 4.]
[3. 3. 3.]
[3. 3. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 3. 4.]
[3. 3. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 4.]
[3. 2. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 3.]
[3. 3. 4.]
[3. 3. 3.]
[3. 3. 3.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[3. 3. 4.]
[3. 3. 4.]
[4. 4. 4.]
[4. 3. 4.]
[4. 3. 3.]
[4. 3. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[4. 3. 4.]
[3. 3. 4.]
[4. 4. 4.]
[3. 3. 4.]
[3. 3. 4.]

[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 3. 5.]
[4. 5. 4.]
[4. 3. 4.]
[4. 4. 4.]
[4. 4. 4.]
[5. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 5.]
[4. 4. 3.]
[4. 4. 5.]
[4. 4. 3.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 4.]
[4. 4. 2.]
[4. 4. 5.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 4.]
[4. 4. 5.]
[4. 4. 3.]
[4. 3. 5.]
[4. 4. 4.]

In [94]:
pred=np.ones(len(all_pred_rounded))
for i in range (len(all_pred_rounded)):
    if all_pred_rounded[i][0]==all_pred_rounded[i][1]:
        pred[i]=all_pred_rounded[i][0]
    elif all_pred_rounded[i][0]==all_pred_rounded[i][2]:
        pred[i]=all_pred_rounded[i][0]
    elif all_pred_rounded[i][1]==all_pred_rounded[i][2] and all_pred_rounded[i][1]!=-1000:
        pred[i]=all_pred_rounded[i][1]
    else : 
        print(all_pred_rounded[i], all_pred[i])
        pred[i]=all_pred_rounded[i][0]
    if pred[i]>5:
        pred[i]=5
    elif pred[i]<1:
        pred[i]=1

[3. 2. 4.] [2.8668823  2.48098233 3.66211725]
[3. 2. 4.] [2.7954271  2.48064582 3.71203275]
[4. 3. 5.] [3.52443648 3.48619146 4.57163673]
[4. 3. 5.] [3.61848249 3.47155646 4.55794362]
[3. 4. 2.] [3.39489234 3.54554837 2.08663218]
[4. 3. 5.] [3.58225922 3.48085379 4.74244349]
[3. 4. 2.] [3.48493741 3.59788984 2.41649035]
[3. 2. 4.] [2.72747536 2.43734517 3.51345549]
[4. 3. 5.] [3.53747224 3.48214136 4.64654887]
[4. 5. 3.] [4.45594933 4.53756936 3.44946223]
[4. 5. 3.] [4.43338726 4.52644457 2.86709398]
[    5. -1000. -1000.] [    4.66598557 -1000.         -1000.        ]
[2. 1. 3.] [1.73489743 1.48824314 2.87122333]
[3. 4. 2.] [3.44799297 3.7921325  2.3318731 ]
[3. 4. 2.] [3.21025299 3.61028826 2.47056517]
[3. 4. 2.] [3.35217043 3.71397423 2.3396907 ]
[3. 4. 2.] [3.40963875 3.9308975  1.66086277]
[4. 3. 2.] [3.52232577 3.23391545 2.39394068]
[3. 4. 5.] [3.45549869 3.80162428 4.83576158]
[3. 4. 2.] [3.22109854 3.68543942 1.96413373]
[4. 5. 3.] [4.35935289 4.70004252 3.46744355]
[4. 3. 5.]

[3. 2. 4.] [2.65609693 1.98836548 3.52552987]
[3. 2. 4.] [2.66646496 2.24442004 3.61897869]
[3. 2. 4.] [2.67399847 2.42476432 3.90229259]
[    3. -1000. -1000.] [    3.10134265 -1000.         -1000.        ]
[3. 4. 2.] [3.41523636 3.70365265 2.47943247]
[4. 3. 5.] [3.53135907 3.44205085 4.58535985]
[3. 2. 4.] [2.61421861 2.31899695 4.05753833]
[2. 1. 3.] [1.62253047 1.45903158 2.96323564]
[2. 1. 3.] [1.75246856 1.48167119 2.79106265]
[3. 2. 4.] [2.51586317 2.23327912 3.84887391]
[3. 2. 4.] [2.6263817  2.33748448 3.52970602]
[3. 2. 4.] [2.68761025 2.49111046 3.96909801]
[3. 4. 2.] [3.31523025 3.52591693 2.44700273]
[3. 4. 2.] [3.30975642 3.56011439 2.07081244]
[3. 2. 4.] [2.54244006 2.42445891 3.51048513]
[3. 2. 4.] [2.84453459 2.49322311 3.5823917 ]
[4. 5. 3.] [4.38265076 4.5222852  2.98171999]
[3. 2. 4.] [2.59727191 2.39141254 3.65675331]
[3. 2. 4.] [3.03933983 2.26212121 3.79336895]
[4. 5. 3.] [4.33180317 4.5273212  2.90354598]
[4. 5. 3.] [4.34485054 4.61699797 2.75969749]
[4. 5. 3.]

[3. 2. 4.] [2.76900856 2.48478083 3.59668528]
[3. 2. 4.] [2.93002579 2.48884331 3.56120207]
[2. 1. 3.] [1.7959455  1.39211994 2.67132274]
[3. 2. 4.] [2.649422   2.49300414 3.50415243]
[3. 2. 4.] [2.61092024 2.49555797 3.6784795 ]
[3. 2. 4.] [2.60988994 2.3782942  3.71092982]
[4. 3. 5.] [3.65405858 3.49229708 4.66375246]
[4. 3. 5.] [3.68972601 3.47735325 4.95764394]
[4. 3. 5.] [3.69976787 3.44667954 4.93836823]
[3. 4. 2.] [3.27054553 3.53896738 1.98604683]
[4. 3. 5.] [3.72892132 3.45778428 4.65446866]
[3. 2. 4.] [2.77776323 2.45590187 3.71818427]
[3. 4. 2.] [3.3586845  3.55208158 2.17407663]
[3. 2. 4.] [2.72906022 2.4754931  3.56026642]
[    3. -1000. -1000.] [    3.27807228 -1000.         -1000.        ]
[3. 2. 4.] [3.03238569 2.46966423 3.63942931]
[2. 1. 3.] [1.56708234 1.34784808 2.90354301]
[2. 1. 3.] [1.94361976 1.49999137 3.14402595]
[2. 1. 3.] [1.90533602 1.40425764 2.9535965 ]
[2. 1. 3.] [1.71032802 1.38448094 3.06421976]
[4. 5. 3.] [4.47388646 4.64564183 3.17520435]
[    4. -1

[3. 2. 4.] [2.51491808 2.37317638 3.65177237]
[3. 2. 4.] [2.56645016 2.45319592 3.8254409 ]
[3. 2. 4.] [2.83021057 2.47765702 3.66055723]
[3. 2. 4.] [2.6993055  2.37245894 3.59698717]
[3. 2. 4.] [2.52323412 2.30973237 3.51487531]
[3. 2. 4.] [2.6338356  2.35961768 3.96895704]
[2. 3. 4.] [2.28837684 2.58274177 3.65775546]
[3. 2. 4.] [2.54892268 2.32616083 3.9528581 ]
[3. 2. 4.] [2.93361442 2.49746422 3.68818615]
[3. 2. 4.] [2.6454032  2.47548791 3.558353  ]
[4. 5. 3.] [4.47162447 4.58329125 3.48016812]
[3. 4. 2.] [3.42864912 3.6332948  2.35142511]
[    4. -1000. -1000.] [    4.01560455 -1000.         -1000.        ]
[4. 5. 2.] [4.43989672 4.57586391 2.36870123]
[3. 2. 4.] [2.51732465 2.46819171 3.69901961]
[3. 4. 2.] [3.40560588 3.52982624 2.12629861]
[3. 4. 2.] [3.47554319 3.66612539 2.06312227]
[2. 3. 4.] [2.37826226 2.55450965 3.71103405]
[2. 3. 4.] [2.49046879 2.55596044 3.84306085]
[4. 3. 5.] [3.50218757 3.45640566 4.51629955]
[2. 1. 3.] [1.64750078 1.43059788 2.89018529]
[2. 1. 3.]

[3. 2. 4.] [2.55910517 2.01615323 3.50081278]
[3. 2. 4.] [2.71510423 2.33247879 3.52618266]
[3. 2. 4.] [2.74261288 2.4558136  3.53192618]
[2. 1. 3.] [1.77968309 1.37455917 3.10972446]
[2. 1. 3.] [1.62748449 1.28036559 2.79113566]
[2. 1. 3.] [1.52600832 1.15479658 2.78085135]
[3. 2. 4.] [2.71061258 2.3879259  3.5697854 ]
[4. 5. 3.] [4.27718878 4.52059908 3.37677883]
[4. 5. 3.] [4.14870778 4.52632535 3.44874293]
[3. 4. 2.] [3.43546113 3.67210643 2.4986403 ]
[3. 2. 4.] [2.76855156 2.36972091 3.63825887]
[3. 2. 4.] [2.85865914 2.48033183 3.71893029]
[3. 4. 2.] [3.4183922  3.51365216 2.45372247]
[3. 4. 2.] [3.40570175 3.56488965 2.31551972]
[3. 2. 4.] [2.73193313 2.45200395 3.55641632]
[3. 4. 2.] [3.14005297 3.54731795 2.44742312]
[3. 2. 4.] [2.60198126 2.47490648 3.58332005]
[4. 3. 5.] [3.65957536 3.48076736 4.53213505]
[2. 1. 3.] [1.91259973 1.42733336 2.7943395 ]
[2. 1. 3.] [1.73834239 1.4763489  3.23939029]
[3. 2. 4.] [2.68237644 2.46281456 3.63569369]
[3. 2. 4.] [2.68132892 2.39534692 

[4. 5. 3.] [4.49518359 4.79502584 2.97267664]
[4. 5. 3.] [4.35520304 4.55661482 3.44962625]
[4. 5. 3.] [4.26171466 4.53954643 3.33931772]
[3. 4. 5.] [3.44344119 3.50074678 4.51526082]
[4. 5. 3.] [4.36527194 4.65850822 3.49526001]
[4. 5. 3.] [4.25177993 4.53962833 3.41364697]
[4. 5. 3.] [4.41726788 4.59063404 3.26783618]
[4. 5. 3.] [4.49161449 4.64634088 3.40730554]
[4. 5. 3.] [4.40813179 4.56628452 3.02940279]
[4. 5. 3.] [4.45345875 4.61915663 3.25476142]
[4. 5. 3.] [4.42200426 4.62003984 3.35153013]
[4. 5. 3.] [4.39102059 4.52441002 3.21436024]
[    4. -1000. -1000.] [    4.48366243 -1000.         -1000.        ]
[3. 2. 4.] [2.51943058 2.37161898 3.70819655]
[5. 6. 4.] [5.19182172 5.50688428 4.29908126]
[4. 3. 5.] [3.53322028 3.48789756 4.51163449]
[3. 2. 4.] [2.77787578 2.43335609 3.70733395]
[4. 3. 5.] [3.72752858 3.44316006 4.51722177]
[3. 2. 4.] [2.84133427 2.49157739 4.15280943]
[3. 2. 4.] [2.63629568 2.26292092 4.06329631]
[3. 4. 2.] [3.48564713 3.53451848 2.36451394]
[3. 2. 4.]

[3. 2. 4.] [2.59276454 2.25651477 3.5165302 ]
[3. 2. 4.] [2.68467973 2.39404596 3.67009248]
[3. 2. 4.] [2.72878056 2.40660701 3.63691327]
[3. 2. 4.] [2.56891702 2.48851665 3.51854521]
[3. 2. 4.] [2.92692922 2.42645976 3.52977732]
[3. 2. 4.] [2.59455131 2.26019452 3.50448254]
[3. 2. 4.] [2.77342805 2.2296112  3.69808773]
[3. 2. 4.] [2.65066337 2.17235938 3.70185926]
[3. 2. 4.] [2.84695304 2.38468751 3.56069811]
[3. 2. 4.] [2.55738005 2.29730697 3.60645467]
[3. 2. 4.] [2.85187282 2.43045063 3.62604499]
[3. 2. 4.] [2.81136341 2.41627445 3.62534721]
[3. 2. 4.] [2.77690835 2.37232012 3.5699084 ]
[3. 2. 4.] [3.12238457 2.48964975 3.60095775]
[3. 2. 4.] [2.50431171 2.27412278 3.52388362]
[3. 2. 4.] [2.78812231 2.39057516 3.62759421]
[3. 2. 4.] [2.84846479 2.34218748 3.56595743]
[3. 2. 4.] [2.64710769 2.3134989  3.50383922]
[3. 2. 4.] [2.64795155 2.33736178 3.56108792]
[3. 2. 4.] [2.84341547 2.41327116 3.51876369]
[3. 2. 4.] [2.87267849 2.46814211 3.50933275]
[3. 2. 4.] [2.76855973 2.22073046 

[    4. -1000. -1000.] [    3.90630504 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.69727235 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.13965442 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.36771514 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.02201781 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.93809672 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.02715425 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.64219952 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.17686361 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.94318982 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.06587157 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.39277679 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.02461597 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.22567162 -1000.         -1000.        ]
[    4. -1000. -1000

[    4. -1000. -1000.] [    3.9252346 -1000.        -1000.       ]
[    4. -1000. -1000.] [    3.79693267 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.70263027 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.09129482 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.10744418 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.87643295 -1000.         -1000.        ]
[    3. -1000. -1000.] [    3.38240821 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.65661544 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.88538329 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.1000564 -1000.        -1000.       ]
[    3. -1000. -1000.] [    3.30973516 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.7190132 -1000.        -1000.       ]
[    4. -1000. -1000.] [    3.91353859 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.60001052 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3

[    4. -1000. -1000.] [    3.7243259 -1000.        -1000.       ]
[    4. -1000. -1000.] [    3.59474661 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.86331047 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.25777975 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.65637521 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.84270633 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.09851476 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.89845324 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.29247511 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.55138267 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.85146803 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.10847052 -1000.         -1000.        ]
[    4. -1000. -1000.] [    3.60090244 -1000.         -1000.        ]
[    4. -1000. -1000.] [    4.10790785 -1000.         -1000.        ]
[    4. -1000. -1000.] 

In [ ]:
#Cross validation
param_grid = { 
    'n_epochs': [100],
    'lr_all': [0.001],
    'reg_all': np.linspace(0.08,0.2,10)
} 
cv=5
algorithms = [SVDpp]
n_jobs=2 #number of cpu used, be carefull if you put too much the os freeze
#définition du model selon les paramètres, attention à importer dans la première cell les autres algos

#  https://surprise.readthedocs.io/en/stable/model_selection.html#surprise.model_selection.search.GridSearchCV
algos=[]
for algorithm in algorithms:
    gs = model_selection.GridSearchCV(algorithm, param_grid, measures=['rmse'], cv=cv,n_jobs=n_jobs,joblib_verbose=100 )
    gs.fit(data) # les calculs se font actuellement ici 
    print(gs.best_params)
    print(gs.best_score)
    algos.append([gs.best_estimator["rmse"],gs.best_score]) #choix de l'algo selon l'erreur, touuuut inclus
    [algo[0].fit(data.build_full_trainset()) for algo in algos] # ici on va train notre algo sur le dataset complet, sans cv car les paramètres sont optimaux
 

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  2.3min
[Parallel(n_jobs=2)]: Done   2 tasks      | elapsed:  2.4min
[Parallel(n_jobs=2)]: Done   3 tasks      | elapsed:  4.5min
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  4.6min
[Parallel(n_jobs=2)]: Done   5 tasks      | elapsed:  6.7min


[(<surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7f81cb4e33c8>, 1)]


In [117]:
"""
#Reload this shit every time because we modify it after
df2 = pd.read_csv("Datasets/sample_submission.csv")

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")


array = np.ones((df2.shape[0],1))
fails=0
for i in df2.iterrows():
    sum_=0
    sum_coef=0
    for algo in algos:
        try :
            sum_+= algo[0].estimate(int(i[1][2]), int(i[1][3]))*algo[1]
            sum_coef+=algo[1]
        except Exception as e: 
            print(e)
            fails+=1
            print("number error :", fails)
            sum_+=3
            sum_coef=1
    val=(int(round(sum_/sum_coef)))
    if val>5:
        val=5
    elif val<=0:
        val=1
    
    array[i[0]]= val
    if i[0]%10000==0:
        print(i[0])
"""

0
User and/or item is unkown.
number error : 1
10000
20000
30000
40000
50000
User and/or item is unkown.
number error : 2
60000
User and/or item is unkown.
number error : 3
User and/or item is unkown.
number error : 4
70000
User and/or item is unkown.
number error : 5
80000
90000
100000
110000
120000
130000
User and/or item is unkown.
number error : 6
140000
150000
User and/or item is unkown.
number error : 7
160000
170000
180000
User and/or item is unkown.
number error : 8
User and/or item is unkown.
number error : 9
190000
User and/or item is unkown.
number error : 10
User and/or item is unkown.
number error : 11
200000
User and/or item is unkown.
number error : 12
210000
User and/or item is unkown.
number error : 13
User and/or item is unkown.
number error : 14
220000
230000
240000
User and/or item is unkown.
number error : 15
250000
User and/or item is unkown.
number error : 16
260000
270000
User and/or item is unkown.
number error : 17
280000
User and/or item is unkown.
number err

User and/or item is unkown.
number error : 590
User and/or item is unkown.
number error : 591
User and/or item is unkown.
number error : 592
User and/or item is unkown.
number error : 593
User and/or item is unkown.
number error : 594
User and/or item is unkown.
number error : 595
User and/or item is unkown.
number error : 596
User and/or item is unkown.
number error : 597
User and/or item is unkown.
number error : 598
User and/or item is unkown.
number error : 599
User and/or item is unkown.
number error : 600
User and/or item is unkown.
number error : 601
User and/or item is unkown.
number error : 602
User and/or item is unkown.
number error : 603
User and/or item is unkown.
number error : 604
User and/or item is unkown.
number error : 605
User and/or item is unkown.
number error : 606
User and/or item is unkown.
number error : 607
User and/or item is unkown.
number error : 608
User and/or item is unkown.
number error : 609
User and/or item is unkown.
number error : 610
User and/or i

number error : 1067
User and/or item is unkown.
number error : 1068
User and/or item is unkown.
number error : 1069
User and/or item is unkown.
number error : 1070
User and/or item is unkown.
number error : 1071
User and/or item is unkown.
number error : 1072
User and/or item is unkown.
number error : 1073
User and/or item is unkown.
number error : 1074
User and/or item is unkown.
number error : 1075
User and/or item is unkown.
number error : 1076
User and/or item is unkown.
number error : 1077
User and/or item is unkown.
number error : 1078
User and/or item is unkown.
number error : 1079
User and/or item is unkown.
number error : 1080
User and/or item is unkown.
number error : 1081
User and/or item is unkown.
number error : 1082
User and/or item is unkown.
number error : 1083
User and/or item is unkown.
number error : 1084
User and/or item is unkown.
number error : 1085
User and/or item is unkown.
number error : 1086
User and/or item is unkown.
number error : 1087
User and/or item is 

In [118]:
fails

1549

La différence entre algo.predict et algo.estimate est que predict devrait être utilisé sur des valeurs que l'on connait d'avance, par exemple si on sait que le user 5 pour le film 12 a voté 3, on peut rentrer ces valeurs et voir "l'erreur"
algo.estimate fait littéralement ce qu'on veut pour le projet, c'est à dire estimer. Pas à 100% sur sur de ça ici mais c'est ce qui me parait le plus juste. Pour l'instant on boucle sur algo.estimate mais il y a peut etre mieux à faire 
il faudra round la valeur la remettre dans le pred.csv etc

In [169]:
array

NameError: name 'array' is not defined

In [170]:
df2.Prediction = array_svd_round

In [172]:
df2[df2.Prediction==5] #uncomment to see the shape of how many of x value there are

,Id,Prediction
838,r37_c3,5.0
840,r43_c3,5.0
854,r181_c3,5.0
859,r225_c3,5.0
863,r270_c3,5.0
...,...,...
1175535,r312_c1000,5.0
1175559,r475_c1000,5.0
1176049,r4162_c1000,5.0
1176524,r7373_c1000,5.0


In [174]:
df2.head

<bound method NDFrame.head of                   Id  Prediction
0             r37_c1         4.0
1             r73_c1         4.0
2            r156_c1         4.0
3            r160_c1         4.0
4            r248_c1         4.0
...              ...         ...
1176947  r9974_c1000         4.0
1176948  r9977_c1000         3.0
1176949  r9978_c1000         3.0
1176950  r9982_c1000         4.0
1176951  r9996_c1000         4.0

[1176952 rows x 2 columns]>

In [173]:
df2 = df2.drop(columns=["user", "item"])

KeyError: "['user' 'item'] not found in axis"

In [175]:
df2.to_csv("Datasets/submission.csv", index=False)